# China's intermediates trade with Asia

How much intermediates does China export to Asia? How much does it import from Asia?

For these calculations, Turkey is counted under Asia while Russia is not. 

In [1]:
import numpy as np
import pandas as pd
import duckdb

## Load MRIO data and aggregate to country level

In [2]:
root = 'ADB-MRIO'
years = np.arange(2017, 2022+1)

df = pd.DataFrame()

for year in years:

    df_t = duckdb.sql(
        f"""
        SELECT {year} as t, exporter, importer, SUM(z) as z
        FROM (
            SELECT 
                SUBSTRING(C0, 1, 3) AS exporter, 
                SUBSTRING(C0, 5, 3) AS i, 
                SUBSTRING(C1, 1, 3) AS importer, 
                SUBSTRING(C1, 5, 3) AS j, 
                z
            FROM (
                UNPIVOT (
                    SELECT C0, COLUMNS('[A-Z]+_c\d')
                    FROM read_parquet('../data/mrio/{root}/{root}-{year}.parquet')
                    WHERE C0 NOT LIKE 'r%' AND C0 NOT LIKE 'trs'
                ) subquery
                ON COLUMNS(* EXCLUDE(C0))
                INTO NAME C1 VALUE z
            )
            WHERE SUBSTRING(C0, 1, 3) != SUBSTRING(C1, 1, 3)
        )
        GROUP BY exporter, importer
        """
    ).df()

    df = pd.concat([df, df_t])

In [3]:
df

,t,exporter,importer,z
0,2017,AUS,AUT,113.933554
1,2017,AUS,BEL,560.912424
2,2017,AUS,BGR,263.933308
3,2017,AUS,BRA,1410.773001
4,2017,AUS,CAN,722.510148
...,...,...,...,...
5251,2022,RoW,SVK,10056.386488
5252,2022,MLD,CAM,0.304624
5253,2022,ECU,EGY,0.484926
5254,2022,ECU,TUR,43.824204


## Identify members of Asia

In [4]:
countries = pd.read_excel('../data/raw/countries.xlsx')
countries.dropna(subset=['mrio'], inplace=True)
countries['continent'].fillna('Rest of the world', inplace=True)
countries['code'].replace('ROW', 'RoW', inplace=True)

countries['asia'] = countries['continent'].str.contains('Asia').astype(int)
countries = countries[['code', 'name', 'asia']]

In [5]:
df = df.merge(countries, left_on=['exporter'], right_on=['code'])
df.rename(columns={'asia': 'exporter_asia'}, inplace=True)
df = df.merge(countries, left_on=['importer'], right_on=['code'])
df.rename(columns={'asia': 'importer_asia'}, inplace=True)
df = df[['t', 'exporter', 'exporter_asia', 'importer', 'importer_asia', 'z']]

## Calculate China–Asia intermediate flows and save

In [10]:
china_to_asia = df[(df['exporter'] == 'PRC') & (df['importer_asia'] == 1)]
china_to_asia = china_to_asia.groupby(['t'])['z'].sum().reset_index()
china_to_asia.rename(columns={'z': 'china_to_asia'}, inplace=True)

china_total_intexports = df[df['exporter'] == 'PRC']
china_total_intexports = china_total_intexports.groupby(['t'])['z'].sum().reset_index()
china_total_intexports.rename(columns={'z': 'china_total_intexports'}, inplace=True)

asia_total_intexports = df[(df['exporter_asia'] == 1) & (df['exporter'] != 'PRC')]
asia_total_intexports = asia_total_intexports.groupby(['t'])['z'].sum().reset_index()
asia_total_intexports.rename(columns={'z': 'asia_total_intexports'}, inplace=True)

asia_to_china = df[(df['importer'] == 'PRC') & (df['exporter_asia'] == 1)]
asia_to_china = asia_to_china.groupby(['t'])['z'].sum().reset_index()
asia_to_china.rename(columns={'z': 'asia_to_china'}, inplace=True)

china_total_intimports = df[df['importer'] == 'PRC']
china_total_intimports = china_total_intimports.groupby(['t'])['z'].sum().reset_index()
china_total_intimports.rename(columns={'z': 'china_total_intimports'}, inplace=True)

asia_total_intimports = df[(df['importer_asia'] == 1) & (df['importer'] != 'PRC')]
asia_total_intimports = asia_total_intimports.groupby(['t'])['z'].sum().reset_index()
asia_total_intimports.rename(columns={'z': 'asia_total_intimports'}, inplace=True)

In [12]:
DF = pd.merge(china_to_asia, china_total_intexports)
DF = DF.merge(asia_total_intexports)
DF = DF.merge(asia_to_china)
DF = DF.merge(china_total_intimports)
DF = DF.merge(asia_total_intimports)
DF.to_csv('../data/adhoc/china_asia_intermediates.csv', index=False)

In [13]:
DF

,t,china_to_asia,china_total_intexports,asia_total_intexports,asia_to_china,china_total_intimports,asia_total_intimports
0,2017,387247.912497,1.153618e+06,3.024663e+06,485744.695344,1.328822e+06,3.026141e+06
1,2018,468511.106253,1.297952e+06,3.525110e+06,730715.544458,1.934337e+06,3.425869e+06
2,2019,514857.795815,1.206930e+06,3.280024e+06,715604.774445,1.874520e+06,3.289411e+06
3,2020,500861.874753,1.185497e+06,2.966012e+06,693426.447621,1.778993e+06,2.940655e+06
4,2021,669768.190564,1.668736e+06,3.840554e+06,875286.067432,2.340541e+06,3.812193e+06
5,2022,793836.770933,1.903865e+06,4.256897e+06,868727.621532,2.479236e+06,4.551621e+06
